# Project: Handwritten Digit Recognition (MNIST)
_Last updated: 2025-08-14 08:34 UTC_

**Learning Outcomes**
- Data loading & preprocessing
- Baseline vs CNN
- Evaluation (confusion matrix, ROC)
- Error analysis & improvements

## 0. Setup

In [ ]:

import numpy as np, matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline


## 1. Load Data

In [ ]:

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
plt.imshow(x_train[0], cmap='gray'); plt.title(int(y_train[0]))


## 2. Baseline: Logistic Regression / MLP (✅ Exercise)

In [ ]:

# TODO: Build a simple MLP baseline
from tensorflow import keras
from tensorflow.keras import layers

x_train_flat = x_train.reshape((-1, 28*28))
x_test_flat = x_test.reshape((-1, 28*28))

baseline = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
baseline.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
baseline.fit(x_train_flat, y_train, epochs=3, batch_size=128, validation_split=0.1)


## 3. CNN (✅ Exercise)

In [ ]:

cnn = keras.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

x_train_c = x_train[..., None]
x_test_c = x_test[..., None]
cnn.fit(x_train_c, y_train, epochs=3, batch_size=128, validation_split=0.1)


## 4. Evaluation & Confusion Matrix

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
y_pred = np.argmax(cnn.predict(x_test_c, verbose=0), axis=1)
cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=False, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted"); plt.ylabel("True");


## 5. Error Analysis (✅ Exercise)
- Show 20 most confident wrong predictions
- Try data augmentation and compare


In [ ]:

# TODO: visualize errors
wrong_idx = np.where(y_pred != y_test)[0][:20]
fig, axes = plt.subplots(4,5, figsize=(10,8))
for ax, idx in zip(axes.ravel(), wrong_idx):
    ax.imshow(x_test[idx], cmap='gray')
    ax.axis('off')
plt.suptitle("First 20 errors")
